In [2]:
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements()

Cloning into 'notebooks'...
remote: Enumerating objects: 515, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 515 (delta 139), reused 126 (delta 122), pack-reused 354
Receiving objects: 100% (515/515), 28.61 MiB | 28.23 MiB/s, done.
Resolving deltas: 100% (246/246), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [3]:
from utils import *
setup_chapter()

Using transformers v4.11.3
Using datasets v1.16.1


# Dataset

In [4]:
## IOB 2 data format
import pandas as pd

In [5]:
tokens = "Jeff Dean is a computer scientist at Google in California".split()

In [6]:
lbls = ["B-PER", "I-PER", "O", "O", "O", "O", "O", "B-ORG", "O", "B-LOC"]

In [9]:
# df = pd.DataFrame([tokens,lbls], index = ['Tokens','Tags'])


In [10]:
df

,0,1,2,3,4,5,6,7,8,9
Tokens,Jeff,Dean,is,a,computer,scientist,at,Google,in,California
Tags,B-PER,I-PER,O,O,O,O,O,B-ORG,O,B-LOC


In [12]:
pd.DataFrame(list(zip(tokens,lbls)), columns = ['Tokens','Labels'])

,Tokens,Labels
0,Jeff,B-PER
1,Dean,I-PER
2,is,O
3,a,O
4,computer,O
5,scientist,O
6,at,O
7,Google,B-ORG
8,in,O
9,California,B-LOC


In [13]:
## Load Dataset, XTREME has multiple domains. First lets know about the PAN_X subsets within XTREME

In [14]:
from datasets import get_dataset_config_names

In [15]:
config_xtreme = get_dataset_config_names("xtreme")

Downloading:   0%|          | 0.00/9.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.1k [00:00<?, ?B/s]

In [17]:
len(config_xtreme)

183

In [18]:
panx_subsets = [s for s in config_xtreme if s.startswith("PAN")]

In [19]:
panx_subsets

['PAN-X.af',
 'PAN-X.ar',
 'PAN-X.bg',
 'PAN-X.bn',
 'PAN-X.de',
 'PAN-X.el',
 'PAN-X.en',
 'PAN-X.es',
 'PAN-X.et',
 'PAN-X.eu',
 'PAN-X.fa',
 'PAN-X.fi',
 'PAN-X.fr',
 'PAN-X.he',
 'PAN-X.hi',
 'PAN-X.hu',
 'PAN-X.id',
 'PAN-X.it',
 'PAN-X.ja',
 'PAN-X.jv',
 'PAN-X.ka',
 'PAN-X.kk',
 'PAN-X.ko',
 'PAN-X.ml',
 'PAN-X.mr',
 'PAN-X.ms',
 'PAN-X.my',
 'PAN-X.nl',
 'PAN-X.pt',
 'PAN-X.ru',
 'PAN-X.sw',
 'PAN-X.ta',
 'PAN-X.te',
 'PAN-X.th',
 'PAN-X.tl',
 'PAN-X.tr',
 'PAN-X.ur',
 'PAN-X.vi',
 'PAN-X.yo',
 'PAN-X.zh']

In [20]:
## The suffixes are language codes

In [21]:
## To load german, we need de as input

In [22]:
from datasets import load_dataset

In [24]:
load_dataset("xtreme", name = "PAN-X.de")

Downloading:   0%|          | 0.00/234M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
})

In [25]:
### Building dataset according to different languages

In [26]:
from collections import defaultdict
from datasets import DatasetDict

In [27]:
DatasetDict?

In [28]:
defaultdict?

In [29]:
# Returns a dataset dict if key doesn't exist

In [30]:
panx_ch = defaultdict(DatasetDict)

In [31]:
panx_ch

defaultdict(datasets.dataset_dict.DatasetDict, {})

In [32]:
langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

In [34]:
for lang, frac in zip(langs, fracs):
  ds = load_dataset("xtreme", name = f"PAN-X.{lang}")

  ## Shuffle and downsample each split according to spoken proportion
  for split in ds:
    panx_ch[lang][split]  = (ds[split].shuffle(seed=0).select(range(int(frac*ds[split].num_rows))))

  0%|          | 0/3 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17. Subsequent calls will reuse this data.


...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xtreme downloaded and prepared to /root/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/2fc6b63c5326cc0d1f73060649612889b3a7ed8a6605c91cecdbd228a7158b17. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
panx_ch

defaultdict(datasets.dataset_dict.DatasetDict,
            {'de': DatasetDict({
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 12580
                 })
             }),
             'fr': DatasetDict({
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 2290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 2290
                 })
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'la

In [41]:
## Inspecting element
element = panx_ch["de"]["train"][0]

In [58]:
print(element)

{'tokens': ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in',
'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.'], 'ner_tags': [0, 0, 0, 0,
5, 6, 0, 0, 5, 5, 6, 0], 'langs': ['de', 'de', 'de', 'de', 'de', 'de', 'de',
'de', 'de', 'de', 'de', 'de']}


In [59]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags: Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER',
'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], names_file=None, id=None),
length=-1, id=None)
langs: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [61]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature

In [62]:
print(tags)

ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG',
'B-LOC', 'I-LOC'], names_file=None, id=None)


In [66]:
### class label to names
tags.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [67]:
def create_tag_names(batch):
  return {"ner_tags_str":[tags.int2str(idx) for idx in batch["ner_tags"]]}

In [68]:
panx_de = panx_ch["de"].map(create_tag_names)

  0%|          | 0/6290 [00:00<?, ?ex/s]

  0%|          | 0/6290 [00:00<?, ?ex/s]

  0%|          | 0/12580 [00:00<?, ?ex/s]

In [73]:
print(panx_de["train"][10])

{'tokens': ['Als', 'Bestandteil', 'des', 'Chianti', 'ist', 'die', 'Sorte',
'vor', 'allem', 'in', 'der', 'Toskana', 'verbreitet', '.'], 'ner_tags': [0, 0,
0, 5, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0], 'langs': ['de', 'de', 'de', 'de', 'de',
'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de'], 'ner_tags_str': ['O',
'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O']}


In [74]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
['Tokens', 'Tags'])
     

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


#### Distribution of each entity

In [75]:
from collections import Counter

In [76]:
Counter?

In [77]:
split2freqs = defaultdict(Counter)

In [78]:
split2freqs

defaultdict(collections.Counter, {})

In [79]:
panx_de.items()

dict_items([('validation', Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 6290
})), ('test', Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 6290
})), ('train', Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 12580
}))])

In [82]:
split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
          

In [83]:
pd.DataFrame.from_dict(split2freqs, orient="index")
     

,ORG,LOC,PER
validation,2683,3172,2893
test,2573,3180,3071
train,5366,6186,5810


# Multi lingual transformers

Simmilar to monolingual transformers with the difference being these are trained on multiple languages at once. A remarkable thing is that despite receiving no specific instruction, the model can generalize accross all languages. This is called as Cross Linual Transfer.

Can be trained in 3 ways:
1. en: Fine Tune on English and then evaluate on each language's test set.

2. each: fine tune and evaluate on monolingual data to measure per language performance

3. all: fine tune on all the training data to evaluate on each language test set.

# Tokenizers

In [84]:
from transformers import AutoTokenizer

In [85]:
bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [87]:
text = "Jack Sparrow loves New Delhi"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [88]:
bert_tokens

['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'Delhi', '[SEP]']

In [89]:
xlmr_tokens

['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁Delhi', '</s>']

In [91]:
pd.DataFrame(zip(bert_tokens, xlmr_tokens), columns = ['bert','xlmr_roberta'])

,bert,xlmr_roberta
0,[CLS],<s>
1,Jack,▁Jack
2,Spa,▁Spar
3,##rrow,row
4,loves,▁love
5,New,s
6,Delhi,▁New
7,[SEP],▁Delhi
